### Set up the board:
Download the base overlay:

In [7]:
# initialize camera from OpenCV
import cv2 as cv

videoIn = cv.VideoCapture(-1)
while(videoIn.isOpened() == False):
    videoIn = cv.VideoCapture(-1)
    videoIn.set(cv.CAP_PROP_FRAME_WIDTH, 640);
    videoIn.set(cv.CAP_PROP_FRAME_HEIGHT, 480);
    videoIn.set(cv.CAP_V4L);

print("Capture device is open: " + str(videoIn.isOpened()))

Capture device is open: True


In [8]:
# Import the driver Overlay class
from MyBaseOverlay import MyBaseOverlay
Base = MyBaseOverlay('base_to_HDMI.bit')
from pynq.lib.video import *

In [9]:
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = Base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

In [10]:
# monitor (output) frame buffer size
frame_out_w = 1920
frame_out_h = 1080

#frame_out_w = 32
#frame_out_h = 32


# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

In [11]:
x,y,w,h = 300,0,100,40

In [13]:
# Capture webcam video and display to HDMI Output
#import numpy as np
import time
start = time.time()
#for NumOfFrames in range (100): 
while(1):
    ret, frame = videoIn.read()
    if (ret):
        outframe = hdmi_out.newframe()
        crop_frame = frame[0:480, 0:480]
        outframe[0:480,0:640,:] = frame[0:480,0:640,:]        
        cv.rectangle(outframe, (x,y), (x + w, y + h), (0,0,0), -1)
        cv.putText(img=outframe, text="CAT",org=(x+int(w/10),y+int(4*h/5)), fontFace=cv.FONT_HERSHEY_DUPLEX, fontScale=1, color=(255,0,0), thickness=1)
        #outframe[0:32,0:32,:] = crop_frame[0:32,0:32,:]
        #outframe = cv.resize(crop_frame,None,fx=(1/15),fy=(1/15),interpolation=cv.INTER_AREA)
        hdmi_out.writeframe(outframe)
    else:
        #raise RuntimeError("Failed to read from camera.")
        print("Failed to read from camera.")
        videoIn.release()
        videoIn = cv.VideoCapture(-1)
        print("Attempt retry.")
        while(videoIn.isOpened() == False):
            videoIn = cv.VideoCapture(-1)
            print("Retry")
            videoIn.set(cv.CAP_PROP_FRAME_WIDTH, 640);
            videoIn.set(cv.CAP_PROP_FRAME_HEIGHT, 480);
end = time.time()
print("Frames per second:     " + str(50 / (end - start)))

Failed to read from camera.
Attempt retry.
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Retry
Ret

KeyboardInterrupt: 

In [ ]:
videoIn.release()
#hdmi_out.stop()
#del hdmi_out, frame

### Change Resolution
Change the webcam resolution to 1280 x 800 and repeat the frame streaming

In [ ]:
# camera (input) configuration
frame_in_w = 1280
frame_in_h = 720

# initialize camera from OpenCV
videoIn = cv2.VideoCapture(-1)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))

In [ ]:
# check input resolution
ret, frame = videoIn.read()
frame_w = frame.shape[1]
frame_h = frame.shape[0]
print("Webcam Resolution:", frame_w, "x", frame_h)

In [ ]:
start = time.time()
for NumOfFrames in range (50):
    # read next image
    ret, frame = videoIn.read()
    frame.resize(frame_w*frame_h*3)
    
    if (ret):
        outframe = hdmi_out.newframe()
        for y in range (frame_h):
            #outframe[0:480,0:640,:] = frame_webcam[0:480,0:640,:]
            outframe[y*frame_out_w*3:y*frame_out_w*3+frame_w*3] = frame.data[y*frame_w*3:(y+1)*frame_w*3]
        # copy to frame buffer / show on monitor
        hdmi_out.writeframe(outframe)   
    else:
        readError = readError + 1

end = time.time()
print("Frames per second:     " + str(50 / (end - start)))

### Display image on notebook

In [ ]:
# Output webcam image as JPEG
%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np
plt.imshow(frame[:,:,[2,1,0]])
plt.show()

### Image captured using fswebcam
A generic USB camera can then be lugged into the USB hub and this will enumerate as a /video device under /dev folder on PYNQ distro as shown below.

In [ ]:
from PIL import Image as PIL_Image

orig_img_path = '/home/xilinx/webcam.jpg'
!fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2> /dev/null

img = PIL_Image.open('/home/xilinx/webcam.jpg')
img